# Experimentation with querying over test videos

In [5]:
# basic
import math
import os
import pickle
from PIL import Image
import random
import time

# opencv
import cv2

# torch
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

## Config vars

In [15]:
root = '.'
model_store_path = os.path.join(root, 'models')
model_file = os.path.join(model_store_path, 'bdd_fasterrcnn_mobilenet_v3_large_fpn_1622545030.pt')
backbone_model = 'fasterrcnn_mobilenet_v3_large_fpn'
target_labels = ['car', 'traffic sign', 'pedestrian']
num_classes = len(target_labels) + 1

# path to a sample video
video_file_name = 'cabc30fc-e7726578.mov'
test_video_path = os.path.join(root, 'bdd100k_videos_test_00', 'bdd100k', 'videos', 'test', video_file_name)

#print(os.listdir(test_video_path))

## Utility function

In [3]:
def get_fastrcnn_model(num_classes):

    # load the pretrained model
    if backbone_model == 'fasterrcnn_resnet50_fpn':
        model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    elif backbone_model == 'fasterrcnn_mobilenet_v3_large_fpn':
        model = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_fpn(pretrained=True)

    # number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features

    # replace with pretrained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    
    return model

In [32]:
def play_video(video_path):
    # Create a VideoCapture object and read from input file

    cap = cv2.VideoCapture(video_path)
    
    # get fps
    fps = round(cap.get(cv2.CAP_PROP_FPS))
    
    # Check if camera opened successfully
    if (cap.isOpened()== False): 
        print("Error opening video  file")
    
    frame_count = 0
    # Read until video is completed
    while(cap.isOpened()):

        # Capture frame-by-frame
        ret, frame = cap.read()
        if ret == True:

            # Display the resulting frame
            cv2.imshow('Frame', frame)
            frame_count += 1

            # Press Q on keyboard to  exit
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break

        # Break the loop
        else: 
            break
    
    # When everything done, release 
    # the video capture object
    cap.release()

    # Closes all the frames
    cv2.destroyAllWindows()
    
    print('Video info:')
    print(f'fps: {fps}')
    print(f'duration: {frame_count//fps} seconds')
    

In [33]:
play_video(test_video_path)

Video info:
fps: 30
duration: 22


In [ ]:
# get the model definition
model = get_fastrcnn_model(num_classes)
model.load_state_dict(torch.load(model_file))
model.eval()